In [1]:
import getpass
import os
from langchain_groq import ChatGroq

In [2]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

 ········


In [3]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableSequence, RunnableMap, RunnableParallel
from langchain_core.runnables.history import RunnableWithMessageHistory


from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_chroma import Chroma

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [4]:
LLAMA_8B = "llama3-8b-8192"
LLAMA_70B = "llama3-70b-8192"
GEMMA2_9B = "gemma2-9b-it"

model = ChatGroq(model=LLAMA_8B)

In [5]:
llm = model

In [6]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020C3B3A9FD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020C3B3AAB70>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
from langgraph.checkpoint.memory import MemorySaver

In [8]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

In [9]:
from langgraph.graph import START, StateGraph, END
from typing_extensions import List, TypedDict

In [10]:
from pydantic import BaseModel
from collections.abc import Callable
from langchain_core.documents import Document
from typing import Optional, Any, Iterable, Iterator, Generator
from textwrap import dedent

In [11]:
# @tool
def jodo(a: float, b: float) -> float:
    """Adds two numbers."""
    return a + b

# @tool
def kamkaro(a: float, b: float) -> float:
    """Subtracts two numbers."""
    return a - b

# @tool
def prod_bro(a: float, b: float) -> float:
    """Multiplies two numbers."""
    return a * b

# @tool
def gandhi(a: float, b: float) -> float:
    """Divides two numbers."""
    if b == 0:
        return "Cannot divide by zero."
    return a / b


tools = [jodo, kamkaro, prod_bro, gandhi]

 - `class collections.abc.Iterator`

$ \hspace{10mm} $ ABC for classes that provide the `__iter__()` and `__next__()` methods.


 - `class collections.abc.Iterable`

$ \hspace{10mm} $ ABC for classes that provide the `__iter__()` method.

 - `class collections.abc.Generator`

$ \hspace{10mm} $ ABC for generator classes that implement the protocol defined in PEP 342 that extends iterators with the `send()`, `throw()` and `close()` methods.
.

### Basic calculator agent

In [11]:
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.runnables.graph import Graph
from langchain_core.runnables.base import Runnable


In [11]:
issubclass(Graph, Runnable)

False

In [12]:
class State(TypedDict):
    question: str
    answer: str
    context: Optional[List[Document]]


class Agent(BaseModel):
    model: BaseChatModel
    tools: List[Callable]
    context: Optional[List[Document]]
    workflow: Any = None


    def _create_agent_workflow(self):
        def bind(state: State):
            llm_with_tools = self.model.bind_tools(tools)
            response = llm_with_tools.invoke(state["question"])
            return {"answer": [response]}

        toolnode = ToolNode(tools)
        tools_cache = {tool.__name__: tool for tool in tools}

        def call_tool(state: State):
            last_message = state["answer"][-1]
            tool_call = last_message.tool_calls[0]
            tool_name = tool_call["name"]
            arguments = tool_call["args"]
        
            if tool_name not in tools_cache:
                output = f"Tool `{tool_name}` is not defined"

            else:
                output = tools_cache[tool_name](**arguments)
        
            return {
                "question": state["question"],
                "answer": str(output),
            }

        graph_builder = StateGraph(State)

        graph_builder.add_node(bind)
        graph_builder.add_node(toolnode)
        graph_builder.add_node(call_tool)
        
        graph_builder.add_edge(START, "bind")
        graph_builder.add_edge("bind", "call_tool")
        self.workflow = graph_builder.compile()


    def run(self, input):
        if not self.workflow:
            print("Initializing workflow")
            self._create_agent_workflow()

        response = self.workflow.invoke(input)
        return response

In [13]:
calculator = Agent(model=model, tools=tools, context=[])

In [14]:
calculator.run({"question": "What if you have 3 and then I take one from you?"})

Initializing workflow


{'question': 'What if you have 3 and then I take one from you?', 'answer': '2'}

### Basic calculator agent

In [15]:
class State(TypedDict):
    question: str
    answer: str

In [19]:
def calculate(state: State):
    llm_with_tools = llm.bind_tools(tool_list)
    response = llm_with_tools.invoke(state["question"])
    return {"answer": [response]}


tools = ToolNode(tool_list)


def call_tool(state: State):
    last_message = state["answer"][-1]
    tool_call = last_message.tool_calls[0]
    function_name = tool_call["name"]
    arguments = tool_call["args"]

    if function_name == "add":
        output = add(**arguments)
    elif function_name == "subtract":
         output = subtract(**arguments)
    elif function_name == "multiply":
        output = multiply(**arguments)
    elif function_name == "divide":
         output = divide(**arguments)
    else:
        output = f"Tool `{function_name}` is not defined"

    return {
        "question": state["question"],
        "answer": str(output),
    }

In [64]:
graph_builder = StateGraph(State)

graph_builder.add_node(calculate)
graph_builder.add_node(tools)
graph_builder.add_node(call_tool)

graph_builder.add_edge(START, "calculate")
graph_builder.add_edge("calculate", "call_tool")
graph = graph_builder.compile()

In [65]:
"What if you have 2 and then you have one more?"

'What if you have 2 and then you have one more?'

In [66]:
response = graph.invoke({"question": "What if you have 3 and then I take one from you?"})
print(response["answer"])

2


In [67]:
response

{'question': 'What if you have 3 and then I take one from you?', 'answer': '2'}

### Basic calculator agent with system prompt

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful calculator. You have the following functions available to you: {functions} \n. When using a function, reply in the following format. \n```tool_code\n{{ function_name: <function name>, arguments: {{ arg1: <value>, arg2: <value> }} }} \n```"),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="functions"),
    ]
)

In [12]:
class CalculatorState(TypedDict):
    messages: List[AIMessage | HumanMessage]
    tool_output: str = None
    output: str = None

In [18]:
tools_dict = {tool.__name__: tool for tool in tools}

In [23]:
def call_llm(state: CalculatorState):
    """Call the LLM to decide next step."""
    runnable = (prompt | llm.bind_tools(tools))
    response = runnable.invoke({"messages": state["messages"], "functions": ['add', 'subtract', 'multiply', 'divide']})
    print(response)
    return {"messages": state["messages"] + [response]}


def check_tool_call(state: CalculatorState):
    """Check if llm is asking to use tool or not."""
    last_message = state["messages"][-1]

    print(last_message)

    if last_message.tool_calls:
        return "call_tool"
    return "user_response"


def call_tool(state: CalculatorState):
    """If llm returned a tool call, call the tool"""
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = tool_call.function.arguments

    if function_name == "add":
        output = add(**arguments)
    elif function_name == "subtract":
         output = subtract(**arguments)
    elif function_name == "multiply":
        output = multiply(**arguments)
    elif function_name == "divide":
         output = divide(**arguments)
    else:
        output = f"Tool `{function_name}` is not defined"

    return {
       "messages": state["messages"] + [AIMessage(content="", tool_call_id=tool_call.id, tool_call_result=str(output))],
        "tool_output": output
    }


def user_response(state: CalculatorState):
    """Return last llm output to user."""
    last_message = state["messages"][-1]
    return {"messages": state["messages"], "output": last_message.content}


In [24]:
workflow = StateGraph(CalculatorState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("check_tool_call", check_tool_call)
workflow.add_node("call_tool", call_tool)
workflow.add_node("user_response", user_response)

workflow.set_entry_point("call_llm")

workflow.add_conditional_edges(
    "call_llm",
    check_tool_call,
    {
        "call_tool": "call_tool",
        "user_response": "user_response"
    },
)

workflow.add_edge("call_tool", "call_llm")
workflow.add_edge("user_response", END)

app = workflow.compile()

In [25]:
input_question = "What if you have 3 and then I take one from you?"

In [26]:
inputs = {"messages": [HumanMessage(content=input_question)]}

response = app.invoke(inputs)

content='```tool-use>\n{\n\t"tool_calls": [\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "add",\n\t\t\t\t"description": "Adds two numbers.",\n\t\t\t\t"parameters": {\n\t\t\t\t\t"a": 3,\n\t\t\t\t\t"b": 1\n\t\t\t\t}\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "subtract",\n\t\t\t\t"description": "Subtracts two numbers.",\n\t\t\t\t"parameters": {\n\t\t\t\t\t"a": 3,\n\t\t\t\t\t"b": 1\n\t\t\t\t}\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "multiply",\n\t\t\t\t"description": "Multiplies two numbers.",\n\t\t\t\t"parameters": {\n\t\t\t\t\t"a": 3,\n\t\t\t\t\t"b": 1\n\t\t\t\t}\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"id": "pending",\n\t\t\t"type": "function",\n\t\t\t"function": {\n\t\t\t\t"name": "divide",\n\t\t\t\t"description": "Divides two numbers.",\n\t\t\t\t"parameters": {\n\t\t\t\t\t"a": 3,\n\t\t\t\t\t"b": 1\n\t\t

In [34]:
print(response["messages"][-1].tool_calls)

[]


### Attempt 1

In [15]:
from langchain_core.messages import BaseMessage
from textwrap import dedent

In [16]:
class AgentState(TypedDict):
    messages: List[Any]
    summary: str

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", dedent("""
        You are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.
        Mention the name of tool that will be used.
        """)),
    ]
)

In [18]:
prompt.format_messages()

[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used. If you are not provided with a question, ask the user to pass a question. \n", additional_kwargs={}, response_metadata={})]

In [81]:
def add_system_prompt(state: AgentState):
    response = llm.invoke(prompt.format_messages())
    print("================================================================================================")
    print(prompt.format_messages())
    print(response)
    print("================================================================================================")
    return {"messages": [response]}


In [82]:
workflow = StateGraph(AgentState)

workflow.add_node(add_system_prompt)

workflow.add_edge(START, "add_system_prompt")
workflow.add_edge("add_system_prompt", END)

app = workflow.compile()

In [83]:
app_0.invoke(
    {"summary": ""}
)

[SystemMessage(content="You are a helpful assistant good at using tools. Mention the name of tool that will be used. If you are not provided with a question, ask the user to pass a question. Use the available tools to resolve user's query", additional_kwargs={}, response_metadata={})]
content="I'm excited to assist you! I have a wide range of tools at my disposal to help resolve your queries. If you have a specific question or problem you'd like to discuss, please feel free to pass it along.\n\nIn the meantime, I'd like to introduce some of the tools I'll be using to help you. These include:\n\n* Google (for searching and gathering information)\n* Wikipedia (for in-depth research and reference)\n* Microsoft Excel (for data analysis and calculations)\n* Adobe Acrobat (for document editing and creating PDFs)\n* Chrome DevTools (for debugging and troubleshooting web pages)\n* And many more!\n\nSo, please go ahead and ask your question, and I'll do my best to provide a helpful and accurate

{'messages': [AIMessage(content="I'm excited to assist you! I have a wide range of tools at my disposal to help resolve your queries. If you have a specific question or problem you'd like to discuss, please feel free to pass it along.\n\nIn the meantime, I'd like to introduce some of the tools I'll be using to help you. These include:\n\n* Google (for searching and gathering information)\n* Wikipedia (for in-depth research and reference)\n* Microsoft Excel (for data analysis and calculations)\n* Adobe Acrobat (for document editing and creating PDFs)\n* Chrome DevTools (for debugging and troubleshooting web pages)\n* And many more!\n\nSo, please go ahead and ask your question, and I'll do my best to provide a helpful and accurate response using these tools.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 56, 'total_tokens': 211, 'completion_time': 0.129166667, 'prompt_time': 0.010797433, 'queue_time': 0.018757674000000002, 'total_tim

### Attempt 2

In [38]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant good at using tools. Mention the name of tool that will be used. Use the available tools to resove user's query"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [39]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [40]:
prompt.format_messages(messages=[("user", query)])

[SystemMessage(content="You are a helpful assistant good at using tools. Mention the name of tool that will be used. Use the available tools to resove user's query", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='If you have 3, and two are taken, how many are you left with?', additional_kwargs={}, response_metadata={})]

In [44]:
def add_system_prompt(state: AgentState):
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(prompt.format_messages(messages=state["messages"]))
    print("================================================================================================")
    print(response)
    print("================================================================================================")
    return {"messages": [response]}


In [45]:
workflow = StateGraph(AgentState)

workflow.add_node(add_system_prompt)

workflow.add_edge(START, "add_system_prompt")
workflow.add_edge("add_system_prompt", END)

app = workflow.compile()

In [48]:
query = "You get 3 apples from each of your friend, but then, you have six friends?"

app.invoke(
    {
        "messages":[("user", query)],
    }
)

content='' additional_kwargs={'tool_calls': [{'id': 'call_xcnz', 'function': {'arguments': '{"a":3,"b":6}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1299, 'total_tokens': 1371, 'completion_time': 0.06, 'prompt_time': 0.179242322, 'queue_time': 0.019526685999999988, 'total_time': 0.239242322}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0d91df61-0971-4d1b-b2d5-cb59d090057d-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 6}, 'id': 'call_xcnz', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1299, 'output_tokens': 72, 'total_tokens': 1371}


{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xcnz', 'function': {'arguments': '{"a":3,"b":6}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1299, 'total_tokens': 1371, 'completion_time': 0.06, 'prompt_time': 0.179242322, 'queue_time': 0.019526685999999988, 'total_time': 0.239242322}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0d91df61-0971-4d1b-b2d5-cb59d090057d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 6}, 'id': 'call_xcnz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1299, 'output_tokens': 72, 'total_tokens': 1371})]}

In [ ]:
{
    "messages":[("user": query), ],
}

### Attempt 3 - expanded attempt 1

In [24]:
class AgentState(TypedDict):
    messages: List[Any]
    n_queries: int
    summary: str

In [25]:
def add_system_prompt(state: AgentState):
    messages = state["messages"]
    print(messages)

    if not any(isinstance(msg, SystemMessage) for msg in messages):
        messages = prompt.format_messages() + list(messages)

    response = llm.invoke(prompt.format_messages())
    print("===========================================prompt===============================================")
    print(prompt.format_messages())
    print(response)
    print("================================================================================================")
    return {"messages": prompt.format_messages() + [response] + state["messages"]}


def bind(state: AgentState):
    llm_with_tools = llm.bind_tools(tools)
    print("======================================current state=============================================")
    print(state["messages"])
    print("================================================================================================")
    response = llm_with_tools.invoke(prompt.format_messages(messages=state["messages"]))
    state["n_queries"] += 1
    print("========================================response================================================")
    print(response)
    print("================================================================================================")
    return {"messages": state["messages"] + [response], "n_queries": (state["n_queries"] + 1)}


toolnode = ToolNode(tools)
tools_cache = {tool.__name__: tool for tool in tools}

def call_tool(state: AgentState):
    print("======================================calling tools=============================================")
    print(state["messages"])
    print(state["messages"][-1].tool_calls)
    print("================================================================================================")
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]
    tool_name = tool_call["name"]
    arguments = tool_call["args"]

    if tool_name not in tools_cache:
        output = f"Tool `{tool_name}` is not defined"

    else:
        output = tools_cache[tool_name](**arguments)

    return {
        "messages": state["messages"] + [output],
        "n_queries": state["n_queries"],
    }

In [31]:
workflow = StateGraph(AgentState)

workflow.add_node(add_system_prompt)
workflow.add_node(bind)
workflow.add_node(toolnode)
workflow.add_node(call_tool)


workflow.add_edge(START, "add_system_prompt")
workflow.add_edge("add_system_prompt", "bind")
workflow.add_edge("bind", "call_tool")
workflow.add_edge("call_tool", END)


app_2 = workflow.compile()

In [32]:
query = "You have six friends. You get 3 apples from each of your friend. How many apples did you get in total?"

response = app_2.invoke(
    {
        "messages":[HumanMessage(content=query, additional_kwargs={}, response_metadata={})],
        "n_queries": 0,
    }
)

[HumanMessage(content='You have six friends. You get 3 apples from each of your friend. How many apples did you get in total?', additional_kwargs={}, response_metadata={})]
===========================================prompt===============================================
[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used.\n", additional_kwargs={}, response_metadata={})]
content="I'm happy to help! I can use various tools to assist with your questions. Please let me know what you need help with, and I'll do my best to provide a solution.\n\nSome of the tools I have available include:\n\n* Web Search (Google): I can use this tool to find information on a wide range of topics.\n* Dictionary: I can use this tool to define words and phrases.\n* Calculator: I can use this tool to perform mathematical calculations.\n* Reasoning: I can use my reasoning skills to help 

In [33]:
response["messages"]

[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used.\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm happy to help! I can use various tools to assist with your questions. Please let me know what you need help with, and I'll do my best to provide a solution.\n\nSome of the tools I have available include:\n\n* Web Search (Google): I can use this tool to find information on a wide range of topics.\n* Dictionary: I can use this tool to define words and phrases.\n* Calculator: I can use this tool to perform mathematical calculations.\n* Reasoning: I can use my reasoning skills to help solve problems and answer questions.\n* Tool Use: I can use various tools like Wikipedia, Stack Overflow, etc.\n\nPlease let me know which tool you would like me to use to answer your question.", additional_kwargs={}, response_metadata={'token_usage': {'compl

In [34]:
response["messages"][-1].tool_calls

AttributeError: 'int' object has no attribute 'tool_calls'

In [30]:
response["n_queries"]

2

In [104]:
u_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", query),
    ]
)
u_prompt.format_messages()

[HumanMessage(content='You get 3 apples from each of your friend. You have six friends?', additional_kwargs={}, response_metadata={})]

### Attempt 4

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", dedent("""
        You are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.
        Mention the name of tool that will be used. Give your answer in one sentence.
        """)),
    ]
)

In [67]:
class AgentState(TypedDict):
    messages: List[Any]
    queries: List[str]
    summary: str

In [76]:
def add_system_prompt(state: AgentState):
    response = llm.invoke(prompt.format_messages())
    print("===========================================prompt===============================================")
    print(prompt.format_messages())
    print(response)
    print("================================================================================================")
    return {"messages": prompt.format_messages() + [response]}


def bind(state: AgentState):
    llm_with_tools = llm.bind_tools(tools)
    print("======================================current state=============================================")
    print(state["messages"])
    print("================================================================================================")
    query = state["queries"].pop()
    response = llm_with_tools.invoke(query)
    print("========================================response================================================")
    print(response)
    print("================================================================================================")
    return {"messages": state["messages"] + [HumanMessage(content=query)] + [response], "queries": state["queries"]}


toolnode = ToolNode(tools)
tools_cache = {tool.__name__: tool for tool in tools}

def call_tool(state: AgentState):
    print("======================================calling tools=============================================")
    print(state["messages"])
    print(state["messages"][-1].tool_calls)
    print("================================================================================================")
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]
    tool_name = tool_call["name"]
    arguments = tool_call["args"]

    if tool_name not in tools_cache:
        output = f"Tool `{tool_name}` is not defined"

    else:
        output = tools_cache[tool_name](**arguments)

    return {
        "messages": state["messages"] + [AIMessage(content=str(output))],
        "queries": state["queries"],
    }

def check_continue(state: AgentState):
    if len(state["queries"]):
        return "bind"
    else:
        print("NO MORE QUERIES")
        return END

In [77]:
workflow = StateGraph(AgentState)

workflow.add_node(add_system_prompt)
workflow.add_node(bind)
workflow.add_node(toolnode)
workflow.add_node(call_tool)


workflow.add_edge(START, "add_system_prompt")
workflow.add_edge("add_system_prompt", "bind")
workflow.add_edge("bind", "call_tool")
workflow.add_conditional_edges("call_tool", check_continue)
workflow.add_edge("call_tool", END)


app_3 = workflow.compile()

In [78]:
queries = [
    "You have six friends. You get 3 apples from each of your friend. How many apples did you get in total?",
    "You have 3 balls and then I took two from you. How many do have now?"
]

response = app_3.invoke(
    {
        "messages":[],
        "queries": queries,
    }
)

===========================================prompt===============================================
[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used. Give your answer in one sentence.\n", additional_kwargs={}, response_metadata={})]
content="I'm ready to assist you. What's your question?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 50, 'total_tokens': 63, 'completion_time': 0.010833333, 'prompt_time': 0.009184593, 'queue_time': 0.018638125, 'total_time': 0.020017926}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None} id='run-296f841e-4df6-4612-bcf5-6a4b2f4e9fa7-0' usage_metadata={'input_tokens': 50, 'output_tokens': 13, 'total_tokens': 63}
======================================current state=============================================
[SystemM

In [79]:
[m for m in response["messages"]]

[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used. Give your answer in one sentence.\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm ready to assist you. What's your question?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 50, 'total_tokens': 63, 'completion_time': 0.010833333, 'prompt_time': 0.009184593, 'queue_time': 0.018638125, 'total_time': 0.020017926}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-296f841e-4df6-4612-bcf5-6a4b2f4e9fa7-0', usage_metadata={'input_tokens': 50, 'output_tokens': 13, 'total_tokens': 63}),
 HumanMessage(content='You have 3 balls and then I took two from you. How many do have now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'too

### Attempt 5

In [14]:
from operator import add
from typing import Annotated


class AgentState(TypedDict):
    messages: Annotated[list[Any], add]
    queries: List[str]
    summary: str



In [21]:
def add_system_prompt(state: AgentState):
    response = llm.invoke(prompt.format_messages())
    print("===========================================prompt===============================================")
    print(prompt.format_messages())
    print(response)
    print("================================================================================================")
    return {"messages": prompt.format_messages() + [response]}


def bind(state: AgentState):
    llm_with_tools = llm.bind_tools(tools)
    print("======================================current state=============================================")
    print(state["messages"])
    print("================================================================================================")
    query = state["queries"].pop()
    response = llm_with_tools.invoke(query)
    print("========================================response================================================")
    print(response)
    print("================================================================================================")
    return {"messages": [HumanMessage(content=query)] + [response], "queries": state["queries"]}


toolnode = ToolNode(tools)
tools_cache = {tool.__name__: tool for tool in tools}

def call_tool(state: AgentState):
    print("======================================calling tools=============================================")
    print(state["messages"])
    print(state["messages"][-1].tool_calls)
    print("================================================================================================")
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]
    tool_name = tool_call["name"]
    arguments = tool_call["args"]

    if tool_name not in tools_cache:
        output = f"Tool `{tool_name}` is not defined"

    else:
        output = tools_cache[tool_name](**arguments)

    return {
        "messages": [AIMessage(content=str(output))],
        "queries": state["queries"],
    }

def check_continue(state: AgentState):
    if len(state["queries"]):
        return "bind"
    else:
        print("NO MORE QUERIES")
        return END

In [22]:
workflow = StateGraph(AgentState)

workflow.add_node(add_system_prompt)
workflow.add_node(bind)
workflow.add_node(toolnode)
workflow.add_node(call_tool)


workflow.add_edge(START, "add_system_prompt")
workflow.add_edge("add_system_prompt", "bind")
workflow.add_edge("bind", "call_tool")
workflow.add_conditional_edges("call_tool", check_continue)
workflow.add_edge("call_tool", END)


In [23]:
from langgraph.checkpoint.memory import MemorySaver
checkpointer = MemorySaver()

In [24]:
app_4 = workflow.compile(checkpointer=checkpointer)

In [25]:
queries = [
    "You have six friends. You get 3 apples from each of your friend. How many apples did you get in total?",
    "You have 3 balls and then I took two from you. How many do have now?"
]

config = {"configurable": {"thread_id": "1"}}
response = app_4.invoke(
    input={
        "messages":[],
        "queries": queries,
    },
    config=config,
)

===========================================prompt===============================================
[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used. Give your answer in one sentence.\n", additional_kwargs={}, response_metadata={})]
content="I'm ready to help! What's your question?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62, 'completion_time': 0.01, 'prompt_time': 0.018415525, 'queue_time': 0.15682026300000002, 'total_time': 0.028415525}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None} id='run-73acf0a7-1c57-4f39-b499-20e9d4ab1594-0' usage_metadata={'input_tokens': 50, 'output_tokens': 12, 'total_tokens': 62}
======================================current state=============================================
[SystemMessag

In [26]:
[m for m in response["messages"]]

[SystemMessage(content="\nYou are a helpful assistant good at reasoning and tool use. Use the available tools to answer user's question.\nMention the name of tool that will be used. Give your answer in one sentence.\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm ready to help! What's your question?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62, 'completion_time': 0.01, 'prompt_time': 0.018415525, 'queue_time': 0.15682026300000002, 'total_time': 0.028415525}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-73acf0a7-1c57-4f39-b499-20e9d4ab1594-0', usage_metadata={'input_tokens': 50, 'output_tokens': 12, 'total_tokens': 62}),
 HumanMessage(content='You have 3 balls and then I took two from you. How many do have now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_cal

### Adding conversation summary

Using [MessagesState](https://github.com/langchain-ai/langgraph/blob/main/libs/langgraph/langgraph/graph/message.py#L263)

In [13]:
from langgraph.graph import MessagesState

In [54]:
class AgentState(MessagesState):
    queries: List[str]
    summary: str

In [64]:
def add_system_prompt(state: AgentState):
    response = llm.invoke(prompt.format_messages())
    print("===========================================prompt===============================================")
    print(prompt.format_messages())
    print(response)
    print("================================================================================================")
    return {"messages": prompt.format_messages() + [response]}


def bind(state: AgentState):
    llm_with_tools = llm.bind_tools(tools)
    print("======================================current state=============================================")
    print(state["messages"])
    print("================================================================================================")
    query = state["queries"].pop()
    response = llm_with_tools.invoke(query)
    print("========================================response================================================")
    print(response)
    print("================================================================================================")
    return {
        "messages": [HumanMessage(content=query)] + [response],
        "queries": state["queries"],
        "summary": state["summary"],
    }


toolnode = ToolNode(tools)
tools_cache = {tool.__name__: tool for tool in tools}

def call_tool(state: AgentState):
    print("======================================calling tools=============================================")
    print(state["messages"])
    print(state["messages"][-1].tool_calls)
    print("================================================================================================")
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]
    tool_name = tool_call["name"]
    arguments = tool_call["args"]

    if tool_name not in tools_cache:
        output = f"Tool `{tool_name}` is not defined"

    else:
        output = tools_cache[tool_name](**arguments)

    return {
        "messages": [AIMessage(content=str(output))],
        "queries": state["queries"],
        "summary": state["summary"],
    }


def summarize_conversation(state: AgentState):
    summary = state.get("summary", "")
    if summary:
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)

    print("====================================summarization response======================================")
    print(response)
    print("================================================================================================")
    
    return {
        "summary": response.content,
        "messages": messages + [response],
        "queries": state["queries"],
    }


def check_summarization(state: AgentState):
    if len(state["queries"]):
        return "bind"
    else:
        print("NO MORE QUERIES")
        return END


def check_continue(state: AgentState):
    n_msg = len(state["messages"])
    if n_msg % 6 == 0 and n_msg != 0:
        return "summarize_conversation"

    if len(state["queries"]):
        return "bind"
    else:
        print("NO MORE QUERIES")
        return END


In [65]:
workflow = StateGraph(AgentState)
checkpointer = MemorySaver()

# workflow.add_node(add_system_prompt)
workflow.add_node(bind)
workflow.add_node(toolnode)
workflow.add_node(call_tool)
workflow.add_node(summarize_conversation)


workflow.set_entry_point()  # Equivalent to calling `add_edge(START, key)`.

workflow.add_edge(START, "bind")
# workflow.add_edge("add_system_prompt", "bind")
workflow.add_edge("bind", "call_tool")
workflow.add_conditional_edges("call_tool", check_continue)
workflow.add_conditional_edges("summarize_conversation", check_summarization)
workflow.add_edge("call_tool", END)


In [66]:
app_5 = workflow.compile(checkpointer=checkpointer)

In [69]:
queries = [
    "You have six friends. You get 3 apples from each of your friend. How many apples did you get in total?",
    "You have 3 balls and then I took two from you. How many do have now?",
    "You were ten years old 5 years ago. How old are you now?",
    "You have 10 pens and five students. If you distribute them equally, how much each of them get?",
]

config = {"configurable": {"thread_id": "1"}}
response = app_5.invoke(
    input={
        "messages":[],
        "queries": queries,
        "summary": "",
    },
    config=config,
)

======================================current state=============================================
[HumanMessage(content='You have 3 balls and then I took two from you. How many do have now?', additional_kwargs={}, response_metadata={}, id='321323cc-aca6-444c-92f8-940db9c6d3f0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t5ts', 'function': {'arguments': '{"a":3,"b":-2}', 'name': 'jodo'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1274, 'total_tokens': 1347, 'completion_time': 0.060833333, 'prompt_time': 0.159345903, 'queue_time': 0.02059138699999999, 'total_time': 0.220179236}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6a4aa925-04ab-40b6-bf15-0713ccfe7fca-0', tool_calls=[{'name': 'jodo', 'args': {'a': 3, 'b': -2}, 'id': 'call_t5ts', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1274, 'output_tokens': 73, 'total

In [70]:
response["messages"]

[HumanMessage(content='You have 3 balls and then I took two from you. How many do have now?', additional_kwargs={}, response_metadata={}, id='321323cc-aca6-444c-92f8-940db9c6d3f0'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t5ts', 'function': {'arguments': '{"a":3,"b":-2}', 'name': 'jodo'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1274, 'total_tokens': 1347, 'completion_time': 0.060833333, 'prompt_time': 0.159345903, 'queue_time': 0.02059138699999999, 'total_time': 0.220179236}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6a4aa925-04ab-40b6-bf15-0713ccfe7fca-0', tool_calls=[{'name': 'jodo', 'args': {'a': 3, 'b': -2}, 'id': 'call_t5ts', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1274, 'output_tokens': 73, 'total_tokens': 1347}),
 AIMessage(content='1', additional_kwargs={}, response_metadata={}, id='a66f36

In [40]:
not 0

True

In [65]:
[1,2,3,400,4].pop()

4

In [ ]:

tools = [
    Tool(
        name="add",
        func=add,
        description="Adds two numbers.",
    ),
    Tool(
        name="subtract",
        func=subtract,
        description="Subtracts two numbers.",
    ),
    Tool(
        name="multiply",
        func=multiply,
        description="Multiplies two numbers.",
    ),
    Tool(
        name="divide",
        func=divide,
        description="Divides two numbers.",
    ),
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful calculator. You have the following functions available to you: {functions} \n. When using a function, reply in the following format. \n```tool_code\n{{ function_name: <function name>, arguments: {{ arg1: <value>, arg2: <value> }} }} \n```"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def agent(messages, agent_scratchpad):
    formatted_messages = prompt.format_messages(
        messages=messages,
        agent_scratchpad=format_to_openai_function_messages(agent_scratchpad),
    )
    response = llm.invoke(formatted_messages)
    return response

output_parser = OpenAIFunctionsAgentOutputParser()

tool_executor = ToolExecutor(tools)

class AgentState:
    messages: List
    agent_scratchpad: List
    tool_output: dict


def call_tool(state):
    """Calls the tool and updates intermediate steps"""
    response = output_parser.parse(state.intermediate_steps.pop())
    
    if response.tool not in ["add", "subtract", "multiply", "divide"]:
        return {"messages": state.messages, "agent_scratchpad":state.agent_scratchpad, "tool_output": None, "intermediate_steps":state.intermediate_steps}
    
    tool_output = tool_executor.invoke(
        {"tool_call": response.tool, "tool_input": response.tool_input}
    )
    
    state.intermediate_steps.append(AIMessage(content=str(tool_output),tool_calls=[])) # Add output as agent message
    return {"messages": state.messages, "agent_scratchpad":state.agent_scratchpad, "tool_output": tool_output, "intermediate_steps":state.intermediate_steps}
    
def end(state):
    return {"messages": state.messages, "agent_scratchpad": state.agent_scratchpad, "tool_output": None, "intermediate_steps": state.intermediate_steps}

workflow = StateGraph(AgentState)
workflow.add_node("agent", agent)
workflow.add_node("call_tool", call_tool)
workflow.add_node("end", end)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
      "call_tool": "call_tool",
      "end": "end"
     }
)
workflow.add_edge("call_tool","agent")

app = workflow.compile()

# 8. Run the Agent
inputs = {"messages": [HumanMessage(content="What is 2 + 2 and then multiply by 10?")] , "agent_scratchpad": [], "intermediate_steps": [], "tool_output": None}
result = app.invoke(inputs)

print("Result: ")
if result["tool_output"] is not None:
    print(result["tool_output"])
else:
    print(result["messages"][-1].content)

In [14]:
tools_dict = {tool.__name__: tool for tool in tools}

NameError: name 'tools' is not defined

In [26]:
tools_dict["add"](1, 3)

4